In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col,max,count,sum,mean,stddev_pop,hour,countDistinct,expr,stddev,window,column
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
import math

# Assignment # 1

In [2]:
# sc.stop()

# Initialization of SparkConf which is required for Spark Context
conf = SparkConf().setAppName('myapp').setMaster('local')

# Initialization of SparkContext
sc = SparkContext().getOrCreate(conf=conf)
sc.setLogLevel("OFF")

# Initialization of SparkSession into spark variable
spark = SparkSession(sc)

21/07/21 19:37:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### How to read a text file in pyspark?

In [3]:
rdd = sc.textFile('a_text_file.txt')

### which options are correct to split rdd containing string type data in pyspark?

In [4]:
rdd = rdd.flatMap(lambda line: line.split(" "))
rdd.collect()

['Hello',
 'world',
 'lol',
 'Yellow',
 'world',
 'lol',
 'Hello',
 'world',
 'lol',
 'Hello',
 'world',
 'lol',
 'Hello',
 'world',
 'lol',
 'Hello',
 'world',
 'lol']

### Identify the mistake in the given code and drop the correct left item into right catagory.

In [5]:
words = sc.parallelize ([
"Big Data",
"Data Science",
"Intro to Web",
"Web Engineering",
"Network Theory",
"Machine Learning",
])

words_map = words.map(lambda x: (x, "a", "b"))
# words_map = words.map(lambda x: (x, "a"))
words_map.collect()

[('Big Data', 'a', 'b'),
 ('Data Science', 'a', 'b'),
 ('Intro to Web', 'a', 'b'),
 ('Web Engineering', 'a', 'b'),
 ('Network Theory', 'a', 'b'),
 ('Machine Learning', 'a', 'b')]

### What would be the output of this code?

In [6]:
words = sc.parallelize(["Big Data", "Data Science", "Intro to Web",  "Web Engineering",  "Network Theory", "Machine Learning", ])
words_map = words.filter(lambda x: "Data" in x)
words_map.collect()

['Big Data', 'Data Science']

### Reading CSV file from the same folder where this notebook file exists

In [7]:
df = spark.read.csv('summary.csv', inferSchema=True, header=True)
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'int')]

### Find the maximum number of flights from Origin country to Destination country by reading flightdata

In [8]:
df.select(max(col('count')).alias('Max Flights')).show()

+-----------+
|Max Flights|
+-----------+
|     370002|
+-----------+



### Count those rows where number of flights going from Origin country to Destination country are greater than 10 by reading flightdata again

In [9]:
df.where(col('count') > 10).select(count('*').alias('haha')).show()

+----+
|haha|
+----+
| 208|
+----+



### Count total number of flights having destination country name is United States

In [10]:
df.where(col('DEST_COUNTRY_NAME') == 'United States').agg(sum('count').alias('sum'), count('count').alias('count')).show()

+------+-----+
|   sum|count|
+------+-----+
|411352|  125|
+------+-----+



# Assignment 2

### You are asked to select only the following columns: "video_id", "trending_date", "title", "views", into a new dataframe.

In [11]:
dataframe = spark.read.csv('Video_Statistics.csv', header=True, inferSchema=True)
videoStats = dataframe.select("video_id", "trending_date", "title", "views")
videoStats.show()


+--------+-------------+--------------------+-----+
|video_id|trending_date|               title|views|
+--------+-------------+--------------------+-----+
|       1|            4|  Goodbye First Love|  560|
|       2|            7|       Club Sandwich|  451|
|       3|           10|      Situation, The|  773|
|       4|            5|          Mine Games|  165|
|       5|            4|     Laws of Gravity|  559|
|       6|            3|Official Story, T...|  430|
|       7|           12|          Restaurant|  940|
|       8|            2|          Dead Birds|  613|
|       9|            2|         Signal, The|  759|
|      10|           11|You Were Never Lo...|  108|
|      11|            1|        Wholly Moses|  254|
|      12|            8|       Firehouse Dog|  316|
|      13|            2|        House of Wax|  858|
|      14|           11|Half Moon (a.k.a....|  934|
|      15|            4|        Spider-Man 3|  921|
|      16|           10|         Wall Street|  826|
|      17|  

### Now after selecting the columns we are interested in, we would like to create a new column inside "videoStats" dataframe with the name "new" that contains the views divided by 100.

In [12]:
videoStats = videoStats.withColumn('new', col('views')/100)
videoStats.show()

+--------+-------------+--------------------+-----+----+
|video_id|trending_date|               title|views| new|
+--------+-------------+--------------------+-----+----+
|       1|            4|  Goodbye First Love|  560| 5.6|
|       2|            7|       Club Sandwich|  451|4.51|
|       3|           10|      Situation, The|  773|7.73|
|       4|            5|          Mine Games|  165|1.65|
|       5|            4|     Laws of Gravity|  559|5.59|
|       6|            3|Official Story, T...|  430| 4.3|
|       7|           12|          Restaurant|  940| 9.4|
|       8|            2|          Dead Birds|  613|6.13|
|       9|            2|         Signal, The|  759|7.59|
|      10|           11|You Were Never Lo...|  108|1.08|
|      11|            1|        Wholly Moses|  254|2.54|
|      12|            8|       Firehouse Dog|  316|3.16|
|      13|            2|        House of Wax|  858|8.58|
|      14|           11|Half Moon (a.k.a....|  934|9.34|
|      15|            4|       

### Please write the line of code to calculate the mean of the views

In [13]:
videoStats.select(mean('views')).show()

+----------+
|avg(views)|
+----------+
|     553.4|
+----------+



### Please write the line of code to calculate the standard deviation of the population of the views

In [14]:
videoStats.select(stddev_pop('views').alias('Standard Deviation')).show()

+------------------+
|Standard Deviation|
+------------------+
| 276.5580228451165|
+------------------+



### Please write the line of code to find the maximum of the views

In [15]:
videoStats.select(max('views').alias('Max Views')).show()

+---------+
|Max Views|
+---------+
|      940|
+---------+



### Group the dataframe by the "trending_date" column and aggregate by the mean of the "views" column. The result should be put in a new dataframe named "videoStatGroup"

In [16]:
videoStatGroup = videoStats.groupby('trending_date').agg(mean('views'))
videoStatGroup.show()

+-------------+-----------------+
|trending_date|       avg(views)|
+-------------+-----------------+
|           12|            940.0|
|            1|            254.0|
|            3|            582.5|
|            5|            165.0|
|            4|            638.0|
|            8|            316.0|
|            7|            316.0|
|           10|            799.5|
|           11|            405.0|
|            2|743.3333333333334|
+-------------+-----------------+



# Assignment 3

In [17]:
# Displaying Dataset
dataframe = spark.read.csv('retail_data.csv', header=True, inferSchema=True)
dataframe.createTempView('retail_data')
dataframe.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

### How many orders did customers perform at which hour? (SQL)

In [18]:
spark.sql("select HOUR(InvoiceDate) AS HOUR, COUNT(DISTINCT(InvoiceNo)) AS Orders from retail_data group by HOUR(InvoiceDate)").show()

+----+------+
|HOUR|Orders|
+----+------+
|  12|    23|
|  13|    12|
|  16|    17|
|  15|    15|
|   9|    18|
|  17|     6|
|   8|     6|
|  10|    12|
|  11|    13|
|  14|    21|
+----+------+



### How frequently was each product bought in different countries? (SQL)

In [19]:
 spark.sql("select Country, StockCode, Count(*) as Count from retail_data group by StockCode, Country").show()

+--------------+---------+-----+
|       Country|StockCode|Count|
+--------------+---------+-----+
|United Kingdom|    22154|    1|
|United Kingdom|    22478|    1|
|United Kingdom|    22844|    3|
|United Kingdom|    22768|    4|
|United Kingdom|    16238|    3|
|United Kingdom|    22916|    3|
|United Kingdom|    21704|    2|
|United Kingdom|        M|    2|
|United Kingdom|   35271S|    1|
|United Kingdom|    70007|    4|
|       Germany|    22242|    2|
|United Kingdom|    72741|    4|
|United Kingdom|   84534B|    1|
|United Kingdom|    22342|    1|
|United Kingdom|   46776B|    1|
|United Kingdom|    21816|    1|
|United Kingdom|    22375|    1|
|United Kingdom|   85232B|    5|
|United Kingdom|   84596E|    1|
|United Kingdom|    21851|    2|
+--------------+---------+-----+
only showing top 20 rows



### How many orders did customers perform at which hour? (Dataframe)

In [20]:
dataframe.select(hour('InvoiceDate').alias('HOUR'), 'InvoiceNo').groupby('HOUR').agg(countDistinct('InvoiceNo')).show()

+----+----------------+
|HOUR|count(InvoiceNo)|
+----+----------------+
|  12|              23|
|  13|              12|
|  16|              17|
|  15|              15|
|   9|              18|
|  17|               6|
|   8|               6|
|  10|              12|
|  11|              13|
|  14|              21|
+----+----------------+



### How frequently was each product bought in different countries? (Dataframe)

In [21]:
#Variant one
dataframe.select('Country', 'StockCode', 'Quantity').groupby('Country', 'StockCode').agg(sum('Quantity').alias('Quantity')).show()

+--------------+---------+--------+
|       Country|StockCode|Quantity|
+--------------+---------+--------+
|United Kingdom|    21125|      12|
|United Kingdom|    21794|       1|
|United Kingdom|    21662|       3|
|          EIRE|   85136C|       2|
|United Kingdom|   84032A|      22|
|United Kingdom|   84086C|       2|
|United Kingdom|   90214H|       2|
|United Kingdom|    22753|       9|
|United Kingdom|    21557|       3|
|United Kingdom|    21743|      15|
|United Kingdom|    22488|      12|
|United Kingdom|    22956|       4|
|        Norway|    22531|      24|
|United Kingdom|    22728|      13|
|United Kingdom|    82551|       3|
|United Kingdom|    22581|       4|
|United Kingdom|    21671|       2|
|United Kingdom|   90177E|       1|
|United Kingdom|   84926A|       4|
|United Kingdom|    20726|       2|
+--------------+---------+--------+
only showing top 20 rows



In [22]:
# Variant two
dataframe.groupby('Country', 'StockCode').agg(sum('Quantity').alias('Quantity')).show()

+--------------+---------+--------+
|       Country|StockCode|Quantity|
+--------------+---------+--------+
|United Kingdom|    21125|      12|
|United Kingdom|    21794|       1|
|United Kingdom|    21662|       3|
|          EIRE|   85136C|       2|
|United Kingdom|   84032A|      22|
|United Kingdom|   84086C|       2|
|United Kingdom|   90214H|       2|
|United Kingdom|    22753|       9|
|United Kingdom|    21557|       3|
|United Kingdom|    21743|      15|
|United Kingdom|    22488|      12|
|United Kingdom|    22956|       4|
|        Norway|    22531|      24|
|United Kingdom|    22728|      13|
|United Kingdom|    82551|       3|
|United Kingdom|    22581|       4|
|United Kingdom|    21671|       2|
|United Kingdom|   90177E|       1|
|United Kingdom|   84926A|       4|
|United Kingdom|    20726|       2|
+--------------+---------+--------+
only showing top 20 rows



### Select from a datacube a slice that contains only items from the United States.

In [23]:
# For slice operation, use where clause on only one column and NOT MORE
dataframe.where(col("Country") == "United States").show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



### Dice Operation - Select a sub-cube from a datacube that contains items from United states and United Kingdom with quantities 0

In [24]:
dataframe.where((col("Country") == "United States") | (col("Country") == "United Kingdom")).where(col("Quantity") == 0).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [25]:
dataframe.where(((col("Country") == "United States") | (col("Country") == "United Kingdom")) & (col("Quantity") == 0)).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



### Which of the following codes peforms a rollup from a datacube?

In [26]:
dataframe.select("StockCode", "Country", (col("Quantity") > 0).alias("positive quantity")).show()

+---------+--------------+-----------------+
|StockCode|       Country|positive quantity|
+---------+--------------+-----------------+
|   85123A|United Kingdom|             true|
|    71053|United Kingdom|             true|
|   84406B|United Kingdom|             true|
|   84029G|United Kingdom|             true|
|   84029E|United Kingdom|             true|
|    22752|United Kingdom|             true|
|    21730|United Kingdom|             true|
|    22633|United Kingdom|             true|
|    22632|United Kingdom|             true|
|    84879|United Kingdom|             true|
|    22745|United Kingdom|             true|
|    22748|United Kingdom|             true|
|    22749|United Kingdom|             true|
|    22310|United Kingdom|             true|
|    84969|United Kingdom|             true|
|    22623|United Kingdom|             true|
|    22622|United Kingdom|             true|
|    21754|United Kingdom|             true|
|    21755|United Kingdom|             true|
|    21777

In [27]:
dataframe.where(col("Quantity") > 0).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

# Assignment 4

### How many instances of each product were sold in each country?

In [28]:
# dataframe.filter((col("StockCode") == 22728) & (col("Country") == "France")).groupBy("Country").pivot("StockCode").sum("Quantity").toPandas()
dataframe.filter("StockCode = 22728 AND Country = 'France'").groupBy("Country").pivot("StockCode").sum("Quantity").toPandas()

,Country,22728
0,France,24


In [29]:
dataframe.groupBy("StockCode").pivot("Country").sum("Quantity").toPandas()

,StockCode,Australia,EIRE,France,Germany,Netherlands,Norway,United Kingdom
0,22728,NaN,NaN,24.0,NaN,NaN,NaN,13.0
1,21259,NaN,NaN,NaN,NaN,NaN,NaN,8.0
2,21889,NaN,24.0,NaN,NaN,NaN,NaN,28.0
3,22121,NaN,NaN,NaN,NaN,NaN,NaN,14.0
4,90022,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
1346,84906,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1347,22492,NaN,36.0,36.0,NaN,NaN,NaN,36.0
1348,17164B,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1349,22311,NaN,NaN,NaN,NaN,NaN,NaN,8.0


### Which employees could convince customers to order products?

In [30]:
# Reading Data
employees = spark.read.csv('employees.csv', header=True, inferSchema=True)
employees.createTempView('employees')

customers = spark.read.csv('customers.csv', header=True, inferSchema=True)
customers.createTempView('customers')

orders = spark.read.csv('orders.csv', header=True, inferSchema=True)
orders.createTempView('orders')

In [31]:
# We have used toPandas() here for beauty only. In exams, it should be show() instead.
spark.sql("SELECT * FROM employees e LEFT SEMI JOIN orders o ON e.employee_id = o.salesman_id").toPandas()
# employees.join(orders, expr("employee_id = salesman_id"), "left_semi").toPandas()

,employee_id,Name,email,phone,hire_date,manager_id,job_title
0,70,Samson Ortiz,consectetuer.cursus@utodio.net,0845 46 41,OctOct-1616-21212121,57,Lavinia Melendez
1,71,Stephen Dejesus,quis@magnaLoremipsum.com,(01269) 774419,JunJun-2727-22222222,55,Stacy Reilly
2,72,Alvin Weiss,sociis.natoque@natoquepenatibuset.co.uk,0381 994 1063,SepSep-1313-20202020,50,Inga Snow
3,73,Benjamin Reyes,Nullam.scelerisque@turpisvitae.net,076 8045 4732,FebFeb-1818-21212121,52,Libby Weeks
4,74,Amal Gross,aliquam.eu@semconsequat.net,055 1465 7860,NovNov-0707-21212121,56,Marah Dean
5,75,Kato Hoover,sed.sem.egestas@blanditat.com,(01740) 44119,JulJul-0707-22222222,59,Macey Slater
6,76,Yuli Warren,nisi.nibh.lacinia@malesuada.ca,(010256) 78732,FebFeb-1515-21212121,56,Hanna Pugh
7,77,Wade Nichols,urna.justo.faucibus@sapiencursusin.net,(023) 5115 1644,AugAug-2222-20202020,56,Gwendolyn Ramirez
8,78,Drake Morales,quam.elementum@ProinultricesDuis.org,07495 111025,JulJul-2828-20202020,55,Jeanette Phillips
9,79,Silas Barber,vitae@bibendumullamcorperDuis.com,055 5328 8347,AprApr-2222-22222222,52,Yolanda Ramirez


### Which employees could convince most customers to order products?

In [32]:
# OF COURSE YOU SHOULD RUN EITHER OF THEM AND COMMENT OUT THE OTHER ONE

# Note the expr() function used in agg. It is simply sql representation of the same statement
employees.join(orders, expr("employee_id = salesman_id"),"inner") \
.groupBy("employee_id").agg(expr("count(*) AS sales")).orderBy("sales",ascending=False).toPandas()

# expr() functions equivalent query using dataframe at agg.
employees.join(orders, expr("employee_id = salesman_id"),"inner") \
.groupBy("employee_id").agg(count('*').alias('sales')).orderBy("sales",ascending=False).toPandas()

,employee_id,sales
0,83,9
1,84,8
2,76,7
3,79,7
4,72,6
5,87,6
6,82,6
7,80,6
8,89,6
9,86,5


In [33]:
# Same query as above using spark SQL
spark.sql("SELECT e.employee_id, count(*) as sales FROM employees e INNER JOIN orders o ON e.employee_id = o.salesman_id GROUP BY e.employee_id ORDER BY sales DESC").toPandas()

,employee_id,sales
0,83,9
1,84,8
2,76,7
3,79,7
4,72,6
5,87,6
6,82,6
7,80,6
8,89,6
9,86,5


# Assignment 7

### Use the RDD API to identify how many orders each customer is waiting for (this means, status = "Pending"). The customers are identified only by customer id. Sort the resulting RDD by the number of pending orders in descending order. The variable orders is a reference to the DataFrame Orders

In [34]:
orders.rdd.filter(lambda row: row["status"] == "hall") \
.map(lambda row: (row["customer_id"], 1)).reduceByKey(lambda x,y: x+y) \
.sortBy(lambda x:x[1], ascending=False).collect()

[(55, 3), (56, 3), (63, 2)]

# Assignment 8

### Naive Bayes - Use a Naive Bayes classifier to predict whether you should play tennis when it is (rainy, hot, not windy, and humidity normal)

In [35]:
df = spark.read.csv('weather.csv', header=True, inferSchema=True)

In [36]:
# Probability of YES

# P(yes | rainy, hot, normal, not windy)
# = P(yes) * P(rainy | yes) * P(hot | yes) * P(normal | yes) * P(not windy | yes)
count_yes = df.where("Play = 'YES'").count()
p_yes = count_yes / df.count()
p_rainy_yes = df.where("Outlook = 'rainy' AND Play = 'YES'").count() / count_yes
p_hot_yes = df.where("Temperature = 'hot' AND Play = 'YES'").count() / count_yes
p_normal_yes = df.where("Humidity = 'normal' AND Play = 'YES'").count() / count_yes
p_windy_yes = df.where("Windy = False AND Play = 'YES'").count() / count_yes

probability_yes = p_yes * p_rainy_yes * p_hot_yes * p_normal_yes * p_windy_yes
probability_yes

0.021164021164021163

In [37]:
# Probability of NO

# P(no | rainy, hot, normal, not windy)
# P(no) * P(rainy | no) * P(hot | no) * P(normal | no) * P(not windy | no)

count_no = df.where("Play = 'NO'").count()
p_no = count_no / df.count()
p_rainy_no = df.where("Outlook = 'rainy' AND Play = 'NO'").count() / count_no
p_hot_no = df.where("Temperature = 'hot' AND Play = 'NO'").count() / count_no
p_normal_no = df.where("Humidity = 'normal' AND Play = 'NO'").count() / count_no
p_windy_no = df.where("Windy = False AND Play = 'NO'").count() / count_no
probability_no = p_no * p_rainy_no * p_hot_no * p_normal_no * p_windy_no
probability_no

0.004571428571428573

### Logistic Regression - Spark Classifier - Please classify the following animals into either a Mammal or not a Mammal after apply a logistic regression on the provided dataset.

In [38]:
# load the ZOO dataset:
zoo_data=spark.read.csv("zoo.csv",inferSchema=True,header=True)
zoo_data = zoo_data.withColumn("IsMammal", expr("CASE WHEN Type = 1 THEN 1 ELSE 0 END"))

# preprocess dataset using RFormula
preprocessed_data = RFormula(formula= "IsMammal ~ Hair + Feathers + Eggs + Milk + Airborne + Aquatic +" + 
                             " Predator + Toothed + Backbone + Venomous + Fins + Legs+" +
                            "Tail + Domestic + Catsize")

preprocessed_data = preprocessed_data.fit(zoo_data)
preprocessed_data = preprocessed_data.transform(zoo_data)

# split dataset into training and test data
train, test = preprocessed_data.randomSplit([0.7, 0.3])

# configure classifier
lr = LogisticRegression(labelCol="label", featuresCol="features")

# train classifier
fittedLR = lr.fit(train)

# classify test data set
result = fittedLR.transform(preprocessed_data)

# Displaying only the prediction of selected animals
result.select('AnimalName', 'label', 'prediction').where(expr("AnimalName in ('lobster', 'hawk', 'goat', 'crayfish', 'clam', 'hamster')")).toPandas()

# Printing all
result_extracted = result.select("AnimalName", "IsMammal", "prediction")
result_extracted.show(200)

+----------+--------+----------+
|AnimalName|IsMammal|prediction|
+----------+--------+----------+
|  aardvark|       1|       1.0|
|  antelope|       1|       1.0|
|      bass|       0|       0.0|
|      bear|       1|       1.0|
|      boar|       1|       1.0|
|   buffalo|       1|       1.0|
|      calf|       1|       1.0|
|      carp|       0|       0.0|
|   catfish|       0|       0.0|
|      cavy|       1|       1.0|
|   cheetah|       1|       1.0|
|   chicken|       0|       0.0|
|      chub|       0|       0.0|
|      clam|       0|       0.0|
|      crab|       0|       0.0|
|  crayfish|       0|       0.0|
|      crow|       0|       0.0|
|      deer|       1|       1.0|
|   dogfish|       0|       0.0|
|   dolphin|       1|       1.0|
|      dove|       0|       0.0|
|      duck|       0|       0.0|
|  elephant|       1|       1.0|
|  flamingo|       0|       0.0|
|      flea|       0|       0.0|
|      frog|       0|       0.0|
|      frog|       0|       0.0|
|  fruitba

# Assignment 9

### Gussian Naive Bayes

In [39]:
alice = Row(student="Alice", percent_achieved = 70, pass_exam = "Yes")
bob = Row(student="Bob", percent_achieved = 36, pass_exam = "No")
carol = Row(student="Carol", percent_achieved = 95, pass_exam = "Yes")
dan = Row(student="Dan", percent_achieved = 63, pass_exam = "Yes")
eve = Row(student="Eve", percent_achieved = 43, pass_exam = "No")
frank = Row(student="Frank", percent_achieved = 84, pass_exam = "Yes")
grace = Row(student="Grace", percent_achieved = 54, pass_exam = "Yes")
heidi = Row(student="Heidi", percent_achieved = 15, pass_exam = "No")
ivan = Row(student="Ivan", percent_achieved = 21, pass_exam = "No")
judy = Row(student="Judy", percent_achieved = 91, pass_exam = "Yes")
mallory = Row(student="Mallory", percent_achieved = 34, pass_exam = "No")

dataTable = [alice,bob,carol,dan,eve,frank,grace,heidi,ivan,judy,mallory]
df = spark.createDataFrame(dataTable)

df_yes = df.where(col("pass_exam")=="Yes")
df_no =df.where(col("pass_exam")=="No")

mean_yes = df_yes.select(mean("percent_achieved")).first()[0]
mean_no = df_no.select(mean("percent_achieved")).first()[0]

stddev_yes = df_yes.select(stddev("percent_achieved")).first()[0]
var_yes = stddev_yes * stddev_yes

stddev_no = df_no.select(stddev("percent_achieved")).first()[0]
var_no = stddev_no * stddev_no

print("Mean Yes: ", mean_yes)
print("Var Yes: ",var_yes)
print("Mean No: ",mean_no)
print("Var No: ",var_no)

def gaus(x,mean, variance):
    return (1/(math.sqrt(2*math.pi*variance)))*math.exp(-((61-mean)**2)/(2*variance))

print("P(61|yes): ",gaus(61,mean_yes,var_yes))
print("P(61|No): ",gaus(61,mean_no,var_no))

print("P(yes|61): ", (1/6)*0.159)
print("P(No|61): ", (1/5)*0.0009)

Mean Yes:  76.16666666666667
Var Yes:  267.76666666666665
Mean No:  29.8
Var No:  131.69999999999996
P(61|yes):  0.01586679399696767
P(61|No):  0.0008631916482906201
P(yes|61):  0.0265
P(No|61):  0.00018


# Assignment 10

In [40]:
streaming = spark.read.csv('windowing.csv', inferSchema=True, header=True)
withEventTime = streaming.groupBy(window(col("Time"), "15 minutes", "10 minutes")).count()
withEventTime.toPandas()

,window,count
0,"(2021-07-21 08:00:00, 2021-07-21 08:15:00)",2
1,"(2021-07-21 08:20:00, 2021-07-21 08:35:00)",4
2,"(2021-07-21 08:10:00, 2021-07-21 08:25:00)",3
3,"(2021-07-21 08:40:00, 2021-07-21 08:55:00)",1
4,"(2021-07-21 08:30:00, 2021-07-21 08:45:00)",3
5,"(2021-07-21 07:50:00, 2021-07-21 08:05:00)",1


In [41]:
withEventTime = streaming.selectExpr("*", "cast(cast(Time as double)/1000000000" + " as timestamp) as event_time")
# withEventTime.withWatermark("event_time", "5 seconds").show()
withEventTime.printSchema()

root
 |-- Time: string (nullable = true)
 |-- Entering/Leaving: string (nullable = true)
 |-- Number of Persons: integer (nullable = true)
 |-- event_time: timestamp (nullable = true)

